In [1]:
import xarray as xr
import os
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ds = xr.open_dataset('/home/sebastian/Documentos/analisisNETCDF/años/2022/IVT_2022_12.nc')
df = ds.to_dataframe()
# Filtrar valores mayores a 250
df_filtered = df.loc[df['IVT'] > 250]

# Resetear el índice y agregar el número como índice al principio
df_filtered.reset_index(inplace=True)
df_filtered.index += 1

# Renombrar el nombre de la columna de índice
df = df_filtered.rename_axis("Index", axis="columns")

# Imprimir el DataFrame
print(df)



Index                 time  latitude  longitude         IVT
1      2022-12-01 00:00:00     14.50     -90.00  293.175201
2      2022-12-01 00:00:00     14.50     -89.25  316.742737
3      2022-12-01 00:00:00     14.50     -88.50  269.719666
4      2022-12-01 00:00:00     14.50     -87.75  269.299530
5      2022-12-01 00:00:00     14.50     -87.00  323.037354
...                    ...       ...        ...         ...
313010 2022-12-31 18:00:00    -55.25     -65.25  262.513855
313011 2022-12-31 18:00:00    -56.00     -72.75  275.938171
313012 2022-12-31 18:00:00    -56.00     -72.00  291.504761
313013 2022-12-31 18:00:00    -56.00     -71.25  280.507629
313014 2022-12-31 18:00:00    -56.00     -70.50  262.686523

[313014 rows x 4 columns]


In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import math
import xarray as xr
import os


# Convertir la columna de fecha a un objeto de fecha
df['time'] = pd.to_datetime(df['time'])

# Obtener una lista única de fechas
fechas = df['time'].unique()

# Crear una carpeta para guardar los archivos si no existe
carpeta = './datosGraficados'
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Definir la latitud y longitud de referencia (para el punto (-180,-90))
lat_ref = -90
lon_ref = -180

# Radio de la Tierra en kilómetros
R = 6371

# Iterar sobre las fechas y crear un gráfico para cada una
for i, fecha in enumerate(fechas):
    # Seleccionar los datos correspondientes a la fecha actual
    df_subset = df[df['time'] == fecha]

    # Filtrar los objetos azules
    objetos_azules = df_subset

    if objetos_azules.empty:
        continue

    # Obtener los valores de latitud, longitud y campo IVT para los objetos azules
    latitudes_azules = objetos_azules['latitude']
    longitudes_azules = objetos_azules['longitude']
    campo_ivt = objetos_azules['IVT']  # Asegúrate de reemplazar 'IVT' con el nombre correcto

    # Resto del código para crear el gráfico
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    scatter = ax.scatter(longitudes_azules, latitudes_azules, c=campo_ivt, cmap='coolwarm', s=5, transform=ccrs.PlateCarree())
    plt.colorbar(scatter, label='IVT')
    ax.coastlines()
    ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--', xlocs=range(-180,181,10), ylocs=range(-90,91,10))
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray')
    ax.add_feature(cfeature.RIVERS, linewidth=0.5, edgecolor='blue')
    plt.tight_layout()

    # Guardar el gráfico con el nombre de la fecha correspondiente
    fecha_py = pd.to_datetime(fecha).to_pydatetime()
    nombre_archivo = fecha_py.strftime('%Y-%m-%d') + '.png'
    archivo = os.path.join(carpeta, nombre_archivo)
    plt.savefig(archivo, dpi=fig.dpi)

    # Imprimir información adicional de la imagen
    resolucion = (fig.dpi * fig.get_size_inches()).astype(int)
    tamano_pixel = (1 / fig.dpi) * 25.4
    escala = ax.get_xlim()[1] - ax.get_xlim()[0]
    print("Nombre de la imagen:", nombre_archivo)
    print("Resolución de la imagen (ancho x alto):", resolucion)
    print("Tamaño del píxel:", tamano_pixel, "mm")
    print("Escala de la imagen:", escala)
    print("Otras propiedades geoespaciales:")
    print("- Proyección cartográfica:", ax.projection)
    print("- CRS:", ax.projection.proj4_init)
    print("- Extensión espacial (latitud mínima, latitud máxima, longitud mínima, longitud máxima):", ax.get_extent())
    print("- Unidades de medida: grados")

    # Cerrar la figura para liberar memoria
    plt.close(fig)

print("Gráficos guardados.")

Nombre de la imagen: 2022-12-01.png
Resolución de la imagen (ancho x alto): [1000  600]
Tamaño del píxel: 0.254 mm
Escala de la imagen: 70.94999999999999
Otras propiedades geoespaciales:
- Proyección cartográfica: +proj=eqc +ellps=WGS84 +a=6378137.0 +lon_0=0.0 +to_meter=111319.4907932736 +vto_meter=1 +no_defs +type=crs
- CRS: +ellps=WGS84 +a=6378137.0 +proj=eqc +lon_0=0.0 +to_meter=111319.4907932736 +vto_meter=1 +no_defs
- Extensión espacial (latitud mínima, latitud máxima, longitud mínima, longitud máxima): (-93.225, -22.275, -65.825, 18.325)
- Unidades de medida: grados
Nombre de la imagen: 2022-12-01.png
Resolución de la imagen (ancho x alto): [1000  600]
Tamaño del píxel: 0.254 mm
Escala de la imagen: 70.94999999999999
Otras propiedades geoespaciales:
- Proyección cartográfica: +proj=eqc +ellps=WGS84 +a=6378137.0 +lon_0=0.0 +to_meter=111319.4907932736 +vto_meter=1 +no_defs +type=crs
- CRS: +ellps=WGS84 +a=6378137.0 +proj=eqc +lon_0=0.0 +to_meter=111319.4907932736 +vto_meter=1 +no_d